In [2]:
# libraries
import pickle
import numpy as np
from itertools import product
from collections import defaultdict
from IPython.display import clear_output

In [34]:
f = open("training/hansards.36.2.e", "r", encoding="utf8") 
english_sentences = [ ['-NULL-'] + line.strip().lower().split() for line in f.readlines()]
f.close()
f = open("training/hansards.36.2.f", "r", encoding="utf8")
french_sentences = [line.strip().lower().split() for line in f.readlines()]
f.close()

In [7]:
translations = list(zip(english, french))
N = len(translations)
print(N)

231164


In [8]:
## this cell computes the RFEs (takes ~15 mins)

word_pair_counts = defaultdict(lambda: defaultdict(float))

# count the occurences of each french word given the english word
for t, (english, french) in enumerate(translations):
        
    # print percentage    
    clear_output(wait=True)
    print('{:0.0f}'.format(t/N*100), '%')
    
    # create list of possible alignments
    e_length = len(english)
    f_length = len(french)
    alignments = list(product(range(e_length), range(f_length)))
    
    # count word pair occurences
    for e_i, f_i in alignments:
        english_word = english[e_i]
        french_word = french[f_i]
        
        word_pair_counts[english_word][french_word] += 1

rfe = dict(word_pair_counts)

# compute the RFEs (divide the (f|e) word count by the total count of the (e) word)
for english_word, french_word_counts in rfe.items():
    total_occurences = sum(french_word_counts.values())
    for french_word in rfe[english_word]:
        word_count = rfe[english_word][french_word]
        rfe[english_word][french_word] = word_count / total_occurences
        
# save to pickle
pickle.dump(rfe, open('pickles/rfe.pkl', 'wb'))

100 %


In [9]:
# quick check if everything sums (approx) to 1
rfe = pickle.load(open('pickles/rfe.pkl', 'rb'))

not_one = []
for _, probs in rfe.items():
    prob_sum = sum(probs.values())
    if prob_sum != 1.0:
        not_one.append(prob_sum)
not_one = np.array(not_one)
print(len(not_one[not_one < 0.99999999999]) == 0)

True


# IBM1 EM Algorithm

In [ ]:
def alignment_indices(english_sentence, french_sentence):
    e_length = len(english_sentence)
    f_length = len(french_sentence)
    alignments = list(product(range(e_length), range(f_length)))
    return alignments

In [ ]:
def compute_perplexity(theta_dict, aligned_sentences):
    for english_sentence, french_sentence in aligned_sentences:
        alignments = alignment_indices(english_sentence, french_sentence)
        prob = 0.
        for e_i, f_i in alignments:
            # ???
            prob *= theta_dict[french_sentence[f_i]][english_sentence[e_i]]

In [49]:
iterations = 10

french_vocab = list(set([word for sublist in french_sentences[:100] for word in sublist]))
english_vocab = list(set([word for sublist in english_sentences[:100] for word in sublist]))

# initialize theta_0 uniformly
theta_0 = 1 / len(french_vocab)

theta_dict = defaultdict(lambda: defaultdict(lambda:theta_0))

for k in range(iterations):
    print('iteration', k)
    # initialize all counts to 0
    counts_dict = defaultdict(lambda: defaultdict(lambda: 0.))
    total_f_dict = defaultdict(lambda: 0.)
    total_e_dict = defaultdict(lambda: 0.)
    
    # E-step
    for english, french in translations[:100]:
        for french_word in french:
            total_f_dict[french_word] = 0.
            for english_word in english:
                total_f_dict[french_word] += theta_dict[french_word][english_word]
        for french_word in french:
            for english_word in english:
                value = theta_dict[french_word][english_word] / total_f_dict[french_word]
                counts_dict[french_word][english_word] += value
                total_e_dict[english_word] += value
    
    # M-step
    for english_word in english_vocab:
        for french_word in french_vocab:
            theta_dict[french_word][english_word] = \
            counts_dict[french_word][english_word] / total_e_dict[english_word]
            
    # compute perplexity
    #####

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49


In [50]:
import operator

theta_dict['le']
prediction = max(theta_dict['le'].items(), key=operator.itemgetter(1))[0]

print(theta_dict['le']['new'], theta_dict['le']['the'])
print(prediction)

0.4372456312459352 0.7160576192444245
the


# IBM1 Variational Inference

# IBM2